In [13]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-3272  GCE       4                                       RUNNING  us-central1-a


In [14]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [15]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Jan  8 09:42 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [17]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [18]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'que_buck' 
client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    print(b.name)

graphframes.sh
postings_gcp/0_000.bin
postings_gcp/0_posting_locs.pickle
postings_gcp/100_000.bin
postings_gcp/100_posting_locs.pickle
postings_gcp/101_000.bin
postings_gcp/101_posting_locs.pickle
postings_gcp/102_000.bin
postings_gcp/102_posting_locs.pickle
postings_gcp/103_000.bin
postings_gcp/103_posting_locs.pickle
postings_gcp/104_000.bin
postings_gcp/104_posting_locs.pickle
postings_gcp/105_000.bin
postings_gcp/105_posting_locs.pickle
postings_gcp/106_000.bin
postings_gcp/106_posting_locs.pickle
postings_gcp/107_000.bin
postings_gcp/107_posting_locs.pickle
postings_gcp/108_000.bin
postings_gcp/108_posting_locs.pickle
postings_gcp/109_000.bin
postings_gcp/109_posting_locs.pickle
postings_gcp/10_000.bin
postings_gcp/10_posting_locs.pickle
postings_gcp/110_000.bin
postings_gcp/110_posting_locs.pickle
postings_gcp/111_000.bin
postings_gcp/111_posting_locs.pickle
postings_gcp/112_000.bin
postings_gcp/112_posting_locs.pickle
postings_gcp/113_000.bin
postings_gcp/113_posting_locs.pickle

In [19]:
full_path = "gs://wikidata_preprocessed/*"
parquetFile = spark.read.parquet(full_path)
# import json

# with open('queries_train.json', 'rt') as f:
#   queries = json.load(f)

# lst=[]
# i=1
# for q, true_wids in queries.items():
#   if(i==15):
#     break
#   lst+=true_wids
  
#   i+=1
# queries_lst=list(set(lst))

doc_title_pairs = parquetFile.select("title","id").rdd
# filterd_title = doc_title_pairs.filter(lambda x : x[1] in queries_lst).collect()



In [20]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [21]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

22/01/08 09:46:30 WARN org.apache.spark.SparkContext: The path /home/dataproc/inverted_index_gcp.py has been added already. Overwriting of added paths is not supported in the current version.


In [22]:
from inverted_index_gcp import InvertedIndex

**YOUR TASK (10 POINTS)**: Use your implementation of `word_count`, `reduce_word_counts`, `calculate_df`, and `partition_postings_and_write` functions from Colab to build an inverted index for all of English Wikipedia in under 2 hours.

A few notes: 
1. The number of corpus stopwords below is a bit bigger than the colab version since we are working on the whole corpus and not just on one file.
2. You need to slightly modify your implementation of  `partition_postings_and_write` because the signature of `InvertedIndex.write_a_posting_list` has changed and now includes an additional argument called `bucket_name` for the target bucket. See the module for more details.
3. You are not allowed to change any of the code not coming from Colab. 

In [23]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

# PLACE YOUR CODE HERE
def word_count(text, id):
  ''' Count the frequency of each word in `text` (tf) that is not included in 
  `all_stopwords` and return entries that will go into our posting lists. 
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs 
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  # YOUR CODE HERE
  d={}
  for token in tokens:
    if(token not in all_stopwords):
      if(token not in d.keys()):
        d[token]=(id,1)
      else:
        d[token]=(id,d[token][1]+1)
  return list(zip(d.keys(),d.values()))


def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples 
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  # YOUR CODE HERE
  lst=list(unsorted_pl)
  lst.sort(key=lambda x:x[0])
  return lst

def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  # YOUR CODE HERE
  return postings.map(lambda x: (x[0], len(x[1])))

term_total=Counter()
def calculate_total_term(word,pl):
  total=0
  for _,tf in pl:
    total+=tf
  return word,total


def partition_postings_and_write(postings):
  ''' A function that partitions the posting lists into buckets, writes out 
  all posting lists in a bucket to disk, and returns the posting locations for 
  each bucket. Partitioning should be done through the use of `token2bucket` 
  above. Writing to disk should use the function  `write_a_posting_list`, a 
  static method implemented in inverted_index_colab.py under the InvertedIndex 
  class. 
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and 
      offsets its posting list was written to. See `write_a_posting_list` for 
      more details.
  '''
  # YOUR CODE HERE
  buckets=postings.map(lambda x: (token2bucket_id(x[0]), x)).groupByKey().mapValues(lambda x:list(x))
  bucketsP=buckets.map(lambda x:InvertedIndex.write_a_posting_list(x,bucket_name))
  return bucketsP

In [43]:
# time the index creation time
# t_start = time()
# word counts map
word_counts = doc_title_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
# filtering postings and calculate df
# postings_filtered = postings.filter(lambda x: len(x[1])>50)
w2df = calculate_df(postings)
w2df_dict = w2df.collectAsMap()
totals=postings.map(lambda x:calculate_total_term(x[0],x[1]))
# partition posting lists and write out
_ = partition_postings_and_write(postings).collect()
# index_const_time = time() - t_start

[]


In [46]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='indexTitle'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)

In [47]:
print(super_posting_locs["migraine"])

[('70_000.bin', 351288)]


Putting it all together

In [48]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
inverted.term_total=totals.collectAsMap()
# write the global stats out
inverted.write_index('.', 'index_title')
# upload to gs
index_src = "index_title.pkl"
index_dst = f'gs://{bucket_name}/indexTitle/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://index_title.pkl [Content-Type=application/octet-stream]...
\ [1 files][ 90.6 MiB/ 90.6 MiB]                                                
Operation completed over 1 objects/90.6 MiB.                                     


In [49]:
!gsutil ls -lh $index_dst

 90.56 MiB  2022-01-08T10:05:50Z  gs://que_buck/indexTitle/index_title.pkl
TOTAL: 1 objects, 94954424 bytes (90.56 MiB)
